# Spatial-temporal patterns of aggressive behaviors. 

## A case study Bogotá, Colombia

In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.decomposition  import PCA
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

/home/jorge/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/jorge/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/jorge/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/jorge/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/jorge/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate 

In [2]:
# PARAMETERS
path_prj  = '~/Dropbox/proyectos/crime2020/'
path_data = '~/data/crime/'
sz_stime  = 336                                    # size of time serie 84 days x 4 jornadas
M         = sz_stime//2                            # number of samples extract from fourier t

In [3]:
# LOAD DATASET
dfnuse = pd.read_csv(path_data +'nuse_110220.csv', encoding='utf-8', error_bad_lines=False)
print('nuse shape:',dfnuse.shape)

/home/jorge/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nuse shape: (2268394, 21)


In [12]:
# Dataframe Nuse filtered and sorted
def data_crime(dfn):
    dfo = dfn[['LONGITUD', 'LATITUD', 'LOCALIDAD', 'COD_UPZ', 'UPZ', 'FECHA','MES','ANIO']].sort_values(by=['COD_UPZ'])
    dfo = dfo[ ((dfo['COD_UPZ'] !=  'ND')  & (dfo['COD_UPZ']  != 'UPZ999'))]
    dfo.reset_index(inplace=True)
    dfo = dfo.rename(columns={'index':'INDEX_TIME'})

    sj = pd.to_datetime(dfo['FECHA'])
    hj = 6*(sj.dt.hour//6)
    dfo['JORNADA'] = [x.replace(hour=y, minute=0, second=0) for x,y in zip(sj, hj)]

    dfo['JORN']  = pd.to_datetime(dfo['FECHA']).dt.hour//6
    dfo['DSEM']  = pd.to_datetime(dfo['FECHA']).dt.dayofweek
    dfo['ESDIA'] = [0 if (x==0 or x==3) else 1 for x in dfo['JORN'] ]
    dfo['NSEM']  = pd.to_datetime(dfo['FECHA']).dt.week
    dfo['FSEM']  = [0 if x<4 else 1 for x in dfo['DSEM']]
    return dfo

# resume of UPZ data
def data_upz(dfc):
    dfcx = dfc.groupby(['COD_UPZ', 'UPZ'])[['LATITUD', 'LONGITUD']].mean()
    dfcx['NUM'] = dfc.groupby(['COD_UPZ', 'UPZ'])['FECHA'].count()
    dfcx.reset_index(inplace=True)
    dfcx.rename(columns={'LATITUD':'CXLAT', 'LONGITUD':'CXLON'}, inplace=True)
    dfout = dfcx[['COD_UPZ', 'UPZ', 'NUM', 'CXLAT', 'CXLON']].copy()
    return dfout
                
def get_upz(supz, col='UPZ'):
    s1 = dfupz[dfupz['COD_UPZ']==supz][col]
    return s1.values[0]

# resume of mes_UPZ data
def data_mon_upz(dfc, save=False):
    dfcx = dfc.groupby(['COD_UPZ', 'UPZ', 'MES'])[['LATITUD', 'LONGITUD']].mean()
    dfcx['NUM']    = dfc.groupby(['COD_UPZ', 'UPZ', 'MES'])['LATITUD'].count()
    dfcx.reset_index(inplace=True)
    dfcx.rename(columns={'LATITUD':'CXLAT', 'LONGITUD':'CXLON'}, inplace=True)
    dfout = dfcx[['COD_UPZ', 'UPZ', 'MES', 'NUM', 'CXLAT', 'CXLON']].copy()
    

# group data by jornada definition
def data_upz_jornada(dfc):
    dfout = dfc.pivot_table(index   = ['JORNADA'], 
                            columns = ['COD_UPZ'],
                            values  =  'UPZ', 
                            aggfunc =  'count', 
                            fill_value=0).unstack().to_frame().rename(columns={0:'EVENTS'})
    dfout.reset_index(inplace=True)
    dfout['MES']  = dfout['JORNADA'].dt.month
    dfout['ANIO'] = dfout['JORNADA'].dt.year
    return dfout

# count monthly events by upz
def data_month(dfin):
    dfout = dfin.groupby(['ANIO', 'MES'])['COD_UPZ'].count().to_frame()
    dfout.reset_index(inplace=True)
    dfout.rename(columns={'COD_UPZ':'NUM_U'}, inplace=True)
    return dfout


dfcrime = data_crime(dfnuse)
print('CRIME shape:', dfcrime.shape)
print('Columns dfcrime', dfcrime.columns)
print('--'*40)


dfupz = data_upz(dfcrime)
szu   = len(dfupz)
lupz  = dfupz['COD_UPZ'].to_list()
print('UPZ shape:', dfupz.shape)
print('Columns dfupz', dfupz.columns)
print('--'*40)


dfuj = data_upz_jornada(dfcrime)
print('UPZ_JORNADA shape:', dfuj.shape)
print('dataframe: UPZ[%d] x JORNADA(dias:2154x4):'%szu)
print('Columns dfuj', dfuj.columns)
print('--'*40)


dfmon = data_month(dfuj)
szm = len(dfmon)
print('MONTH shape:', dfmon.shape)
print('Columns dfmon', dfmon.columns)
print('--Init month:', dfmon.iloc[0,1],  dfmon.iloc[0,0]) 
print('--Last month:', dfmon.iloc[-1,1], dfmon.iloc[-1,0]) 
print('--'*40)


CRIME shape: (2245405, 15)
Columns dfcrime Index(['INDEX_TIME', 'LONGITUD', 'LATITUD', 'LOCALIDAD', 'COD_UPZ', 'UPZ',
       'FECHA', 'MES', 'ANIO', 'JORNADA', 'JORN', 'DSEM', 'ESDIA', 'NSEM',
       'FSEM'],
      dtype='object')
--------------------------------------------------------------------------------
UPZ shape: (115, 5)
Columns dfupz Index(['COD_UPZ', 'UPZ', 'NUM', 'CXLAT', 'CXLON'], dtype='object')
--------------------------------------------------------------------------------
UPZ_JORNADA shape: (990840, 5)
dataframe: UPZ[115] x JORNADA(dias:2154x4):
Columns dfuj Index(['COD_UPZ', 'JORNADA', 'EVENTS', 'MES', 'ANIO'], dtype='object')
--------------------------------------------------------------------------------
MONTH shape: (71, 3)
Columns dfmon Index(['ANIO', 'MES', 'NUM_U'], dtype='object')
--Init month: 1 2014
--Last month: 11 2019
--------------------------------------------------------------------------------


In [35]:
# contar ceros por JORNADA y por UPZ
df1 = dfuj[dfuj.EVENTS==0].groupby('COD_UPZ')['JORNADA'].count()
for i, x in df1.items():
    print("%4d  %4d --> (%s) %s"%(x, 8616-x, i, get_upz(i)))

2184 6432 --> (UPR1) UPR ZONA NORTE
1226 7390 --> (UPR3) UPR RIO TUNJUELO
8600   16 --> (UPR4) UPR RIO BLANCO
7147 1469 --> (UPZ1) PASEO DE LOS LIBERTADORES
3203 5413 --> (UPZ10) LA URIBE
1794 6822 --> (UPZ100) GALERIAS
2939 5677 --> (UPZ101) TEUSAQUILLO
 660 7956 --> (UPZ102) LA SABANA
7540 1076 --> (UPZ103) PARQUE SALITRE
5029 3587 --> (UPZ104) PARQUE SIMON BOLIVAR - CAN
7621  995 --> (UPZ105) JARDIN BOTANICO
5582 3034 --> (UPZ106) LA ESMERALDA
4783 3833 --> (UPZ107) QUINTA PAREDES
3505 5111 --> (UPZ108) ZONA INDUSTRIAL
6798 1818 --> (UPZ109) CIUDAD SALITRE ORIENTAL
1757 6859 --> (UPZ11) SAN CRISTOBAL NORTE
5411 3205 --> (UPZ110) CIUDAD SALITRE OCCIDENTAL
4769 3847 --> (UPZ111) PUENTE ARANDA
5310 3306 --> (UPZ112) GRANJAS DE TECHO
5124 3492 --> (UPZ113) BAVARIA
4663 3953 --> (UPZ114) MODELIA
5465 3151 --> (UPZ115) CAPELLANIA
5589 3027 --> (UPZ116) ALAMOS
7228 1388 --> (UPZ117) AEROPUERTO EL DORADO
2419 6197 --> (UPZ12) TOBERIN
1674 6942 --> (UPZ13) LOS CEDROS
3986 4630 --> (UPZ14) US

In [36]:
# Calculate and scale the spectrograms 

ftorSigm = 4
def scaleSpec(x, factor=16):
    e = 2.71828
    y = factor*(x-x.mean())/(1+x.std())
    return 1/(1 + e**(-y))

fs  = 4
wk  = np.arange(0, fs//2, fs/sz_stime)

# Espectrograma por upz
spec = np.zeros((szu*szm, M), dtype=float)

# Calculo del índice de inicio de cada mes 
ndx = []
for i, rs in dfmon.iterrows():
    dt = datetime.datetime(rs.ANIO, rs.MES, 1, 0, 0, 0)
    n  = dfuj[dfuj['JORNADA']==dt].index.values
    ndx.append(n[0])
    
k = 0;
for u in lupz:
    dff = dfuj[dfuj['COD_UPZ']==u]
    for n in ndx:
        if n+sz_stime > len(dff.index): 
            k = k+1
            break
        arr = np.array(dff['EVENTS'][n:n+sz_stime])
        ark = np.fft.fft(arr)
        mark = scaleSpec(np.absolute(ark[:M]), ftorSigm)
        if(True in np.isnan(mark)):
            print(u, n, k, mark, arr)
        spec[[k], :] = mark
        k = k+1        

In [ ]:
# SAVE spectrograms 
w = szm-1
for u in range(szu):
    im = spec[u*w:(u+1)*w, :]
    im = im.T
    plt.title(dfupz.iloc[u][0]+':'+ str(dfupz.iloc[u][1]))
    #plt.ylabel('Frequency (2hz, 0hz)')
    #plt.xlabel('Months')
    plt.tick_params(axis='x', which='both',bottom=False, top=False, labelbottom=False)
    plt.tick_params(axis='y', which='both',left=False, right=False, labelleft=False)
    plt.imshow(im)
    plt.imsave('/home/jorge/Dropbox/proyectos/crime2020/results/spectrograms/'+dfupz.iloc[u][1]+'_'+str(u)+'.png', im)
    print('%3d %s \t%s'%(u, dfupz.iloc[u][0], dfupz.iloc[u][1]))
 